In [1]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
print("Number of GPUs available:", torch.cuda.device_count())


PyTorch version: 2.5.1
CUDA available: True
GPU: NVIDIA A100 80GB PCIe MIG 1g.10gb
Number of GPUs available: 1


In [2]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")
print("Number of GPUs available:", torch.cuda.device_count())


PyTorch version: 2.5.1
CUDA available: True
GPU: NVIDIA A100 80GB PCIe MIG 7g.80gb
Number of GPUs available: 1


In [21]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import os



In [15]:
# CIFAR-10 Dataset with 14x14 patches and augmentations
transform = transforms.Compose([
    transforms.RandomResizedCrop(14, scale=(0.08, 1.0)),  # 14x14 patch
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomGrayscale(p=0.2),
    transforms.GaussianBlur(kernel_size=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)


Files already downloaded and verified


In [16]:
class BagSSLModel(nn.Module):
    def __init__(self, embedding_dim=128):
        super(BagSSLModel, self).__init__()
        self.encoder = resnet34(pretrained=False)  # ResNet-34 backbone
        self.encoder.fc = nn.Identity()  # Remove the final classification head

        # Projector: 2 linear layers (4096 → 128)
        self.projector = nn.Sequential(
            nn.Linear(512, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
            nn.Linear(4096, embedding_dim),
            nn.BatchNorm1d(embedding_dim)
        )
    
    def forward(self, x):
        features = self.encoder(x)
        projections = self.projector(features)
        return projections


In [18]:
def contrastive_loss(z1, z2, temperature=0.1):
    # Normalize embeddings
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)

    # Compute similarity matrix
    similarity_matrix = torch.cat([z1, z2]).mm(torch.cat([z1, z2]).T) / temperature

    # Mask for positive pairs
    batch_size = z1.size(0)
    labels = torch.arange(batch_size).cuda()
    logits = similarity_matrix[:batch_size, batch_size:]

    return F.cross_entropy(logits, labels)


In [20]:
# Initialize model, optimizer, and scheduler
model = BagSSLModel(embedding_dim=128).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.3, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=600)

# Training loop
epochs = 600
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for images, _ in train_loader:
        images = images.cuda()

        # Generate two augmented views
        images1, images2 = torch.split(images, images.size(0) // 2)
        
        # Forward pass
        z1 = model(images1)
        z2 = model(images2)

        # Compute loss
        loss = contrastive_loss(z1, z2)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    scheduler.step()
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}")


/home/synaderi/mycondaenvs/my_pytorch_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/synaderi/mycondaenvs/my_pytorch_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/synaderi/mycondaenvs/my_pytorch_env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid 

Epoch [1/600], Loss: 6.8804
Epoch [2/600], Loss: 6.3041
Epoch [3/600], Loss: 6.3307
Epoch [4/600], Loss: 6.3676


Traceback (most recent call last):
  File "/home/synaderi/mycondaenvs/my_pytorch_env/lib/python3.12/multiprocessing/queues.py", line 259, in _feed
    reader_close()
  File "/home/synaderi/mycondaenvs/my_pytorch_env/lib/python3.12/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/home/synaderi/mycondaenvs/my_pytorch_env/lib/python3.12/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


KeyboardInterrupt: 

In [25]:
print(f"Current working directory: {os.getcwd()}")

Current working directory: /home/synaderi


In [26]:
# Set current working directory
new_working_directory = "/Users/shawheennaderi/programming_projects/yc_project_1"
os.chdir(new_working_directory)

# Verify the change
print(f"Current working directory: {os.getcwd()}")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/shawheennaderi/programming_projects/yc_project_1'